In [29]:
import os
import re
import pandas as pd
import yaml
import uuid

###### C3N-05929 test case

In [81]:
bammap = pd.read_csv('../tests/data/pecgs_TN_wgs_bam/input.bammap', sep='\t')
bammap

,sample_name,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
0,C3N-05929.WGS.T,C3N-05929,LSCC,WGS,tumor,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,12345678,BAM,hg38,d3ddd33b-c7a3-4caf-bdd7-c05e98a19d83,storage1
1,C3N-05929.WGS.N,C3N-05929,LSCC,WGS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,12345678,BAM,hg38,7d8f4ede-1911-456e-9eee-2fafe7132f50,storage1


In [82]:
cases = sorted(set(bammap['case']))
inputs = {}
for case in cases:
    data = {
        'wgs': {
        }
    }
    filtered = bammap[bammap['case']==case]
    # check for wxs tumor bam
    f = filtered[[True if e=='WGS' and df=='BAM' and 'tumor' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 1:
        print(f'Warning: more than 1 wgs tumor bams found for case {case}')
    if f.shape[0]:
        data['wgs']['tumor'] = (f['data_path'].to_list()[0], f['UUID'].to_list()[0])
    
    # check for wxs normal bam
    f = filtered[[True if e=='WGS' and df=='BAM' and 'normal' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 1:
        print(f'Warning: more than 1 wgs normal bams found for case {case}')
    if f.shape[0]:
        data['wgs']['normal'] = (f['data_path'].to_list()[0], f['UUID'].to_list()[0])
            
    # if everything is present then accept sample
    if 'tumor' in data['wgs'] and 'normal' in data['wgs']:
        inputs[case] = data
inputs

{'C3N-05929': {'wgs': {'tumor': ('/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/c5d518a3-fc69-4d7b-a719-0aad38abe43a/23fa88f9-9293-406b-bc21-2928ff12bc8b_wgs_gdc_realn.bam',
    'd3ddd33b-c7a3-4caf-bdd7-c05e98a19d83'),
   'normal': ('/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/823dda5b-ff15-4a66-b28b-de0632850623/e3bf416b-3d08-4d1b-bec4-808e55c7788c_wgs_gdc_realn.bam',
    '7d8f4ede-1911-456e-9eee-2fafe7132f50')}}}

In [83]:
run_list = {}
for case, d in inputs.items():
    run_list[case] = {f'{dt}_{st}_bam.filepath': fp
                     for dt, d1 in d.items()
                     for st, (fp, uid) in d1.items()}
    run_list[case].update({f'{dt}_{st}_bam.uuid': uid
                     for dt, d1 in d.items()
                     for st, (fp, uid) in d1.items()})
run_list

{'C3N-05929': {'wgs_tumor_bam.filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/c5d518a3-fc69-4d7b-a719-0aad38abe43a/23fa88f9-9293-406b-bc21-2928ff12bc8b_wgs_gdc_realn.bam',
  'wgs_normal_bam.filepath': '/storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/823dda5b-ff15-4a66-b28b-de0632850623/e3bf416b-3d08-4d1b-bec4-808e55c7788c_wgs_gdc_realn.bam',
  'wgs_tumor_bam.uuid': 'd3ddd33b-c7a3-4caf-bdd7-c05e98a19d83',
  'wgs_normal_bam.uuid': '7d8f4ede-1911-456e-9eee-2fafe7132f50'}}

In [84]:
run_table = pd.DataFrame.from_dict(run_list).transpose()
run_table['case_id'] = run_table.index.to_list()
run_table['run_uuid'] = [str(uuid.uuid4()) for i in range(run_table.shape[0])]
run_table.index = [f'{c}_{u}' for c, u in zip(run_table.index, run_table['run_uuid'])]
run_table.index.name = 'run_id'
run_table

,wgs_normal_bam.filepath,wgs_normal_bam.uuid,wgs_tumor_bam.filepath,wgs_tumor_bam.uuid,case_id,run_uuid
run_id,,,,,,
C3N-05929_1fd2054c-fc78-433d-8f99-32f8e6909db1,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,7d8f4ede-1911-456e-9eee-2fafe7132f50,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,d3ddd33b-c7a3-4caf-bdd7-c05e98a19d83,C3N-05929,1fd2054c-fc78-433d-8f99-32f8e6909db1


In [85]:
filtered = run_table[['case_id', 'run_uuid', 'wgs_normal_bam.filepath', 'wgs_normal_bam.uuid', 'wgs_tumor_bam.filepath', 'wgs_tumor_bam.uuid']]
filtered.to_csv('../tests/data/pecgs_TN_wgs_bam/run_list.txt', sep='\t')


###### MMRF_1250 test case

In [64]:
bammap = pd.read_csv('../tests/data/pecgs_TN_wxs_bam/input.bammap', sep='\t')
bammap

,sample_name,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
0,C3L-00677.WXS.T,C3L-00677,GBM,WXS,tumor,/storage1/fs1/dinglab/Active/Projects/estorrs/...,12345678,BAM,hg38,6e0bde4a-22c1-4e1f-a38a-dff275b61472,storage1
1,C3L-00677.WXS.N,C3L-00677,GBM,WXS,blood_normal,/storage1/fs1/dinglab/Active/Projects/estorrs/...,12345678,BAM,hg38,8919accb-6973-4ec5-b29f-9d895a1713d4,storage1
2,HT191P1-S1H1A3Y3.RNA-seq.R1.T,C3L-00677,GBM,RNA-seq,tumor,/storage1/fs1/dinglab/Active/Projects/estorrs/...,12345678,FASTQ,NaN,2983df85-958b-483c-95e5-2375426879d1,storage1
3,HT191P1-S1H1A3Y3.RNA-seq.R2.T,C3L-00677,GBM,RNA-seq,tumor,/storage1/fs1/dinglab/Active/Projects/estorrs/...,12345678,FASTQ,NaN,f840b94b-eac9-4f87-98b6-51b26b606e1a,storage1


In [72]:
cases = sorted(set(bammap['case']))
inputs = {}
for case in cases:
    data = {
        'wxs': {
        }
    }
    filtered = bammap[bammap['case']==case]
    # check for wxs tumor bam
    f = filtered[[True if e=='WXS' and df=='BAM' and 'tumor' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 1:
        print(f'Warning: more than 1 wxs tumor bams found for case {case}')
    if f.shape[0]:
        data['wxs']['tumor'] = (f['data_path'].to_list()[0], f['UUID'].to_list()[0])
    
    # check for wxs normal bam
    f = filtered[[True if e=='WXS' and df=='BAM' and 'normal' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 1:
        print(f'Warning: more than 1 wxs normal bams found for case {case}')
    if f.shape[0]:
        data['wxs']['normal'] = (f['data_path'].to_list()[0], f['UUID'].to_list()[0])
            
    # if everything is present then accept sample
    if 'tumor' in data['wxs'] and 'normal' in data['wxs']:
        inputs[case] = data
inputs

{'C3L-00677': {'wxs': {'tumor': ('/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam',
    '6e0bde4a-22c1-4e1f-a38a-dff275b61472'),
   'normal': ('/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam',
    '8919accb-6973-4ec5-b29f-9d895a1713d4')}}}

In [73]:
run_list = {}
for case, d in inputs.items():
    run_list[case] = {f'{dt}_{st}_bam.filepath': fp
                     for dt, d1 in d.items()
                     for st, (fp, uid) in d1.items()}
    run_list[case].update({f'{dt}_{st}_bam.uuid': uid
                     for dt, d1 in d.items()
                     for st, (fp, uid) in d1.items()})
run_list

{'C3L-00677': {'wxs_tumor_bam.filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam',
  'wxs_normal_bam.filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam',
  'wxs_tumor_bam.uuid': '6e0bde4a-22c1-4e1f-a38a-dff275b61472',
  'wxs_normal_bam.uuid': '8919accb-6973-4ec5-b29f-9d895a1713d4'}}

In [74]:
run_table = pd.DataFrame.from_dict(run_list).transpose()
run_table['case_id'] = run_table.index.to_list()
run_table['run_uuid'] = [str(uuid.uuid4()) for i in range(run_table.shape[0])]
run_table.index = [f'{c}_{u}' for c, u in zip(run_table.index, run_table['run_uuid'])]
run_table.index.name = 'run_id'
run_table

,wxs_normal_bam.filepath,wxs_normal_bam.uuid,wxs_tumor_bam.filepath,wxs_tumor_bam.uuid,case_id,run_uuid
run_id,,,,,,
C3L-00677_2a9edf1e-76df-4595-9125-fb9b9d919621,/storage1/fs1/dinglab/Active/Projects/estorrs/...,8919accb-6973-4ec5-b29f-9d895a1713d4,/storage1/fs1/dinglab/Active/Projects/estorrs/...,6e0bde4a-22c1-4e1f-a38a-dff275b61472,C3L-00677,2a9edf1e-76df-4595-9125-fb9b9d919621


In [75]:
filtered = run_table[['case_id', 'run_uuid', 'wxs_normal_bam.filepath', 'wxs_normal_bam.uuid', 'wxs_tumor_bam.filepath', 'wxs_tumor_bam.uuid']]
filtered.to_csv('../tests/data/pecgs_TN_wxs_bam/run_list.txt', sep='\t')

###### HT191P1 test case

In [55]:
bammap = pd.read_csv('../tests/data/pecgs_TN_wxs_fq/input.bammap', sep='\t')
bammap

,sample_name,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
0,HT191P1-S1H1A3Y3.WXS.R1.T,HT191P1-S1H1A3Y3,PDAC,WXS,tumor,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,78777884-505c-4b2f-9170-107662c1effa,storage1
1,HT191P1-S1H1A3Y3.WXS.R2.T,HT191P1-S1H1A3Y3,PDAC,WXS,tumor,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,2699158f-25fe-4f19-a7cf-2a4e1cf687b3,storage1
2,HT191P1-S1H1A3Y3.WXS.R1.N,HT191P1-S1H1A3Y3,PDAC,WXS,blood_normal,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,eafb3178-e459-44c0-a374-8e17a8107fce,storage1
3,HT191P1-S1H1A3Y3.WXS.R2.N,HT191P1-S1H1A3Y3,PDAC,WXS,blood_normal,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,9923349f-dc3c-40d4-a097-803c4828f732,storage1
4,HT191P1-S1H1A3Y3.RNA-seq.R1.T,HT191P1-S1H1A3Y3,PDAC,RNA-seq,tumor,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,fb1f4b8a-a56b-4bc6-b059-864c2b066dc8,storage1
5,HT191P1-S1H1A3Y3.RNA-seq.R2.T,HT191P1-S1H1A3Y3,PDAC,RNA-seq,tumor,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,12345678,FASTQ,NaN,80ce8102-e8a4-4529-85b0-701a4dc022ea,storage1


looking for all samples in bammap with all input files present (in this case paired WXS fastqs and rna-seq tumor fastqs)

In [56]:
cases = sorted(set(bammap['case']))
inputs = {}
for case in cases:
    data = {
        'wxs': {
            'tumor': {},
            'normal': {}
        },
        'rna-seq': {
            'tumor': {}
        }
    }
    filtered = bammap[bammap['case']==case]
    
    # check for wxs tumor fastqs
    f = filtered[[True if e=='WXS' and df=='FASTQ' and 'tumor' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 2:
        print(f'Warning: more than 2 wxs tumor fastqs found for case {case}')
    for i, row in f.iterrows():
        if '.r1.' in row['sample_name'].lower():
            data['wxs']['tumor']['R1'] = (row['data_path'], row['UUID'])
            break
    for i, row in f.iterrows():
        if '.r2.' in row['sample_name'].lower():
            data['wxs']['tumor']['R2'] = (row['data_path'], row['UUID'])
            break
            
    # check for wxs normal fastqs
    f = filtered[[True if e=='WXS' and df=='FASTQ' and 'normal' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 2:
        print(f'Warning: more than 2 wxs normal fastqs found for case {case}')
    for i, row in f.iterrows():
        if '.r1.' in row['sample_name'].lower():
            data['wxs']['normal']['R1'] = (row['data_path'], row['UUID'])
            break
    for i, row in f.iterrows():
        if '.r2.' in row['sample_name'].lower():
            data['wxs']['normal']['R2'] = (row['data_path'], row['UUID'])
            break
            
    # check for rna-seq tumor fastqs
    f = filtered[[True if e=='RNA-seq' and df=='FASTQ' and 'tumor' in st.lower() else False
                 for e, st, df in zip(filtered['experimental_strategy'],
                                      filtered['sample_type'], filtered['data_format'])]]
    if f.shape[0] > 2:
        print(f'Warning: more than 2 rna-seq tumor fastqs found for case {case}')
    for i, row in f.iterrows():
        if '.r1.' in row['sample_name'].lower():
            data['rna-seq']['tumor']['R1'] = (row['data_path'], row['UUID'])
            break
    for i, row in f.iterrows():
        if '.r2.' in row['sample_name'].lower():
            data['rna-seq']['tumor']['R2'] = (row['data_path'], row['UUID'])
            break
            
    
    # if everything is present then accept sample
    # dont do in real life bc pipeline is split by input, but easier this way for test case
    if len(data['wxs']['tumor']) == 2 and len(data['wxs']['normal']) == 2 and len(data['rna-seq']['tumor']) == 2:
        inputs[case] = data
inputs

{'HT191P1-S1H1A3Y3': {'wxs': {'tumor': {'R1': ('/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53_L002_R1_001.fastq.gz',
     '78777884-505c-4b2f-9170-107662c1effa'),
    'R2': ('/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53_L002_R2_001.fastq.gz',
     '2699158f-25fe-4f19-a7cf-2a4e1cf687b3')},
   'normal': {'R1': ('/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R1_001.fastq.gz',
     'eafb3178-e459-44c0-a374-8e17a8107fce'),
    'R2': ('/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R2_001.fastq.gz',
     '9923349f-dc3c-40d4-a097-803c4828f732')}},
  'rna-seq': {'tumor': {'R1': ('/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R1.fast

In [57]:
run_list = {}
for case, d in inputs.items():
    run_list[case] = {f'{dt}_{st}_{r}.filepath': fp
                     for dt, d1 in d.items()
                     for st, d2 in d1.items()
                     for r, (fp, uid) in d2.items()}
    run_list[case].update({f'{dt}_{st}_{r}.uuid': uid
                     for dt, d1 in d.items()
                     for st, d2 in d1.items()
                     for r, (fp, uid) in d2.items()})
run_list

{'HT191P1-S1H1A3Y3': {'wxs_tumor_R1.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53_L002_R1_001.fastq.gz',
  'wxs_tumor_R2.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CCAGTAGCGT-ATGTATTGGC_S53_L002_R2_001.fastq.gz',
  'wxs_normal_R1.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R1_001.fastq.gz',
  'wxs_normal_R2.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/wxs/CATTATCGCT-CTTGAAGGTT_S34_L001_R2_001.fastq.gz',
  'rna-seq_tumor_R1.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAGGTGTTAG-TAAGCGCGTG.HFWJGDSXY_AAGGTGTTAG-TAAGCGCGTG_L001_R1.fastq.gz',
  'rna-seq_tumor_R2.filepath': '/scratch1/fs1/dinglab/estorrs/pecgs_resources/test_samples/HT191P1-S1H1A3Y3/rna-seq/ht191p1-s1h1a3y3e1_1.AAG

In [58]:
run_table = pd.DataFrame.from_dict(run_list).transpose()
run_table['case_id'] = run_table.index.to_list()
run_table['run_uuid'] = [str(uuid.uuid4()) for i in range(run_table.shape[0])]
run_table.index = [f'{c}_{u}' for c, u in zip(run_table.index, run_table['run_uuid'])]
run_table.index.name = 'run_id'
run_table

,rna-seq_tumor_R1.filepath,rna-seq_tumor_R1.uuid,rna-seq_tumor_R2.filepath,rna-seq_tumor_R2.uuid,wxs_normal_R1.filepath,wxs_normal_R1.uuid,wxs_normal_R2.filepath,wxs_normal_R2.uuid,wxs_tumor_R1.filepath,wxs_tumor_R1.uuid,wxs_tumor_R2.filepath,wxs_tumor_R2.uuid,case_id,run_uuid
run_id,,,,,,,,,,,,,,
HT191P1-S1H1A3Y3_be6b68e6-365d-4451-8195-f1b3d5bb5519,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,fb1f4b8a-a56b-4bc6-b059-864c2b066dc8,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,80ce8102-e8a4-4529-85b0-701a4dc022ea,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,eafb3178-e459-44c0-a374-8e17a8107fce,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,9923349f-dc3c-40d4-a097-803c4828f732,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,78777884-505c-4b2f-9170-107662c1effa,/scratch1/fs1/dinglab/estorrs/pecgs_resources/...,2699158f-25fe-4f19-a7cf-2a4e1cf687b3,HT191P1-S1H1A3Y3,be6b68e6-365d-4451-8195-f1b3d5bb5519


wxs fastq runlist

In [11]:
filtered = run_table[['case_id', 'run_uuid', 'wxs_normal_R1.filepath', 'wxs_normal_R1.uuid', 'wxs_normal_R2.filepath', 'wxs_normal_R2.uuid', 'wxs_tumor_R1.filepath', 'wxs_tumor_R1.uuid', 'wxs_tumor_R2.filepath', 'wxs_tumor_R2.uuid']]
filtered.to_csv('../tests/data/pecgs_TN_wxs_fq/run_list.txt', sep='\t')

rnaseq fq runlist

In [59]:
filtered = run_table[['case_id', 'run_uuid', 'rna-seq_tumor_R1.filepath', 'rna-seq_tumor_R1.uuid', 'rna-seq_tumor_R2.filepath', 'rna-seq_tumor_R2.uuid']]
filtered.to_csv('../tests/data/pecgs_T_rna_fq/run_list.txt', sep='\t')

if we want the .bams to have correct readgroup ids, etc. then we need to pass some sequencing info. this info is available in the Samplemap.csv from MGI

In [12]:
df = pd.read_csv('/data/pecgs/test_sample/ht191p1-s1h1a3/Samplemap.csv', sep=',')
df

,Instrument Data ID,Flow Cell ID,Lane,Index Sequence,Status,Library Summary,Library Name,Library Type,Sample Full Name,Species Name,Creation Event ID,Creation Date,Completion Date,Gerald Bam Path,Full Path,File1,File2
0,2903151174,HFMFWDSXY,2,AAGAAGTCAC-ACGAATAAGG,completed,2903011195,TWCE-HT110B1-S1H1A3Y3D1_1-lib1,genomic library,TWCE-HT110B1-S1H1A3Y3D1_1,human,177153983,9/13/20 18:58,9/13/20 20:02,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AAGAAGTCAC-ACGAATAAGG_S4_L002_R1_001.fastq.gz,AAGAAGTCAC-ACGAATAAGG_S4_L002_R2_001.fastq.gz
1,2903151239,HFMFWDSXY,2,AAGAGCTCGC-AGTAAGAGTC,completed,2903011186,TWCE-HT110B1-S1H4A3Y1D1_1-lib1,genomic library,TWCE-HT110B1-S1H4A3Y1D1_1,human,177153994,9/13/20 18:58,9/13/20 20:15,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AAGAGCTCGC-AGTAAGAGTC_S5_L002_R2_001.fastq.gz,AAGAGCTCGC-AGTAAGAGTC_S5_L002_R1_001.fastq.gz
2,2903151057,HFMFWDSXY,2,AAGATCCGTC-TTACACCAAC,completed,2903011207,TWCE-HT088B1-S1H4A3Y1D1_1-lib1,genomic library,TWCE-HT088B1-S1H4A3Y1D1_1,human,177153965,9/13/20 18:57,9/13/20 20:01,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AAGATCCGTC-TTACACCAAC_S6_L002_R2_001.fastq.gz,AAGATCCGTC-TTACACCAAC_S6_L002_R1_001.fastq.gz
3,2903151099,HFMFWDSXY,2,AAGCTAGCTG-CTTCCGTGAA,completed,2903011176,TWCE-HT067B1-S1H2A3Y1D1_1-lib1,genomic library,TWCE-HT067B1-S1H2A3Y1D1_1,human,177153968,9/13/20 18:57,9/13/20 20:01,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AAGCTAGCTG-CTTCCGTGAA_S7_L002_R1_001.fastq.gz,AAGCTAGCTG-CTTCCGTGAA_S7_L002_R2_001.fastq.gz
4,2903151285,HFMFWDSXY,2,AATTAAGCGG-ATGGTCAGTA,completed,2903011173,TWCE-HT163B1-S1H6A3Y2D1_1-lib1,genomic library,TWCE-HT163B1-S1H6A3Y2D1_1,human,177154009,9/13/20 18:58,9/13/20 20:16,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AATTAAGCGG-ATGGTCAGTA_S11_L002_R2_001.fastq.gz,AATTAAGCGG-ATGGTCAGTA_S11_L002_R1_001.fastq.gz
5,2903151242,HFMFWDSXY,2,ACTTGCATAC-ACATGAATGC,completed,2903011194,TWCE-HT069B1-S1H4A3Y1D1_1-lib1,genomic library,TWCE-HT069B1-S1H4A3Y1D1_1,human,177154000,9/13/20 18:58,9/13/20 20:16,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,ACTTGCATAC-ACATGAATGC_S15_L002_R1_001.fastq.gz,ACTTGCATAC-ACATGAATGC_S15_L002_R2_001.fastq.gz
6,2903151337,HFMFWDSXY,2,ACTTGGCTTC-GCAGAGCACA,completed,2903011188,TWCE-HT154B1-S1H1A3Y2D1_1-lib1,genomic library,TWCE-HT154B1-S1H1A3Y2D1_1,human,177154022,9/13/20 18:58,9/13/20 20:21,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,ACTTGGCTTC-GCAGAGCACA_S16_L002_R2_001.fastq.gz,ACTTGGCTTC-GCAGAGCACA_S16_L002_R1_001.fastq.gz
7,2903151190,HFMFWDSXY,2,AGCTGAACTA-TGTGTCCAAC,completed,2903011210,TWCE-HT190P1-S1H9A3Y1D1_1-lib1,genomic library,TWCE-HT190P1-S1H9A3Y1D1_1,human,177153989,9/13/20 18:58,9/13/20 20:03,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,AGCTGAACTA-TGTGTCCAAC_S28_L002_R2_001.fastq.gz,AGCTGAACTA-TGTGTCCAAC_S28_L002_R1_001.fastq.gz
8,2903150959,HFMFWDSXY,2,CAATAGAGGC-CTTACGTATC,completed,2903011204,TWCE-HT191P1-S1H2A3Y3D1_1-lib1,genomic library,TWCE-HT191P1-S1H2A3Y3D1_1,human,177153936,9/13/20 18:57,9/13/20 19:49,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,CAATAGAGGC-CTTACGTATC_S32_L002_R1_001.fastq.gz,CAATAGAGGC-CTTACGTATC_S32_L002_R2_001.fastq.gz
9,2903151064,HFMFWDSXY,2,CACATCAAGC-TGTATAGGCT,completed,2903011198,TWCE-HT154B1-S1H3A3Y2D1_1-lib1,genomic library,TWCE-HT154B1-S1H3A3Y2D1_1,human,177153966,9/13/20 18:57,9/13/20 19:59,NaN,/gscmnt/gc13037/lims/2857177/2020-09/csf_17715...,CACATCAAGC-TGTATAGGCT_S35_L002_R2_001.fastq.gz,CACATCAAGC-TGTATAGGCT_S35_L002_R1_001.fastq.gz


In [13]:
data = [
    ['HT191P1-S1H1A3Y3', '419b8098-e4b2-4318-883d-c233cb7e06c8', 'wxs', 'tumor', 'HFMFWDSXY', '2', 'CCAGTAGCGT-ATGTATTGGC', 'TWCE-HT191P1-S1H1A3Y3D1_1-lib1', 'ILLUMINA'],
    ['HT191P1-S1H1A3Y3', '419b8098-e4b2-4318-883d-c233cb7e06c8', 'wxs', 'normal', 'HH7KNDSXY', '1', 'CATTATCGCT-CTTGAAGGTT', 'TWCE-HT191P1-JM1D1_1-lib1', 'ILLUMINA']
]
sq_df = pd.DataFrame(data=data, columns=['sample_id', 'run_uuid', 'experimental_strategy', 'sample_type',
                                        'flowcell', 'lane', 'index_sequencer', 'library_preparation', 'platform'])
sq_df.index = [f'{s}_{u}' for s, u in zip(sq_df['sample_id'], sq_df['run_uuid'])]
sq_df.index.name = 'run_id'
sq_df



,sample_id,run_uuid,experimental_strategy,sample_type,flowcell,lane,index_sequencer,library_preparation,platform
run_id,,,,,,,,,
HT191P1-S1H1A3Y3_419b8098-e4b2-4318-883d-c233cb7e06c8,HT191P1-S1H1A3Y3,419b8098-e4b2-4318-883d-c233cb7e06c8,wxs,tumor,HFMFWDSXY,2,CCAGTAGCGT-ATGTATTGGC,TWCE-HT191P1-S1H1A3Y3D1_1-lib1,ILLUMINA
HT191P1-S1H1A3Y3_419b8098-e4b2-4318-883d-c233cb7e06c8,HT191P1-S1H1A3Y3,419b8098-e4b2-4318-883d-c233cb7e06c8,wxs,normal,HH7KNDSXY,1,CATTATCGCT-CTTGAAGGTT,TWCE-HT191P1-JM1D1_1-lib1,ILLUMINA


In [14]:
sq_df.to_csv('../tests/data/pecgs_TN_wxs_fq/sequencing_info.txt', sep='\t')


###### save defaults

In [30]:
resources_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources'

In [31]:
pecgs_storage1_defaults = {
    # general
    'cpu': 40,
    
    # align dna-seq
    'known_sites': {
        'class': 'File',
        'path': f'{resources_dir}/dnaseq_alignment/dbsnp/00-All.chr.vcf.gz',
        'secondaryFiles': [
            {'class': 'File', 'path': f'{resources_dir}/dnaseq_alignment/dbsnp/00-All.chr.vcf.gz.tbi'}
        ]
    },
    'reference': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'}
        ]
    },
    'platform': 'ILLUMINA',
    
    # cnv
    'common_biallelic': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
    },
    'pool_of_normals': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5'
    },
    'protein_coding_gene': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed'
    },
    'reference_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1'
    },
    'target_interval_list': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list'
    },
    
    # msisensor
    'microsatellite': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/hg38.microsatellite'
    },
    'minimal_homopolymer_size': 1,
    'minimal_microsatellite_size': 1,
    
    # tindaisy
    'clinvar_annotation': {
        'class': 'File',
        'path': f'{resources_dir}/clinvar/GRCh38/clinvar_20200706.vcf.gz'
    },
    'rescue_clinvar': True,
    'rescue_cosmic': True,
    'call_regions': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.callRegions.bed.gz'
    },
    'canonical_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.callRegions.bed'
    },
    'pindel_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/pindel.WES.ini'
    },
    'strelka_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/strelka.WES.ini'
    },
    'varscan_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/varscan.WES.ini'
    },
    'classification_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/filter_config/classification_filter_config.ini'
    },
    'af_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/filter_config/af_filter_config.ini'
    },
    'centromere_bed': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/centromere/ucsc-centromere.GRCh38.bed'
    },
    'tindaisy_vep_cache_gz': {
        'class': 'File',
        'path': f'{resources_dir}/vep/v99/vep-cache.99_GRCh38.tar.gz'
    },
    'tindaisy_vep_cache_version': 99,
    'assembly': 'GRCh38',
    'tindaisy_chrlist': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.d1.vd1.chrlist.txt'
    },
    
    # TinJasmine
    'centromere': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/centromere/ucsc-centromere.GRCh38.bed'
    },
    'tinjasmine_chrlist': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/chrlist/GRCh38.d1.vd1.chrlist-reordered.txt'
    },
    'pindel_config_template': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_config_template/pindel_germline_filter_config.ini'
    },
    'Canonical_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/Canonical_BED/GRCh38.callRegions.bed'
    },
    'ROI_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/ROI_BED/Homo_sapiens.GRCh38.95.allCDS.2bpFlanks.biomart.withCHR.bed'
    },
#     'varscan_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/varscan_filter_config/VLD_FilterVCF-varscan.config.ini'
#     },
#     'pindel_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_filter_config/VLD_FilterVCF-pindel.config.ini'
#     },
#     'gatk_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/gatk_filter_config/VLD_FilterVCF-GATK.config.ini'
#     },
    'tinjasmine_vep_cache_gz': {
        'class': 'File',
        'path': f'{resources_dir}/vep/20220525/vep-cache.100_GRCh38.tar.gz'
    },
    
    # neoscan
    'neoscan_ref_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/neoscan/refseq_hg38_june29'
    },
    'neoscan_bed': {
        'class': 'File',
        'path': f'{resources_dir}/neoscan/refseq_hg38_june29/proteome.bed'
    },
    
    # charger
    'charger_inheritance_gene_list': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/cancer_pred_genes_160genes_011321_curated_forCharGer.txt'
    },
    'charger_pp2_gene_list': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/160cpgs.txt'
    },
    'charger_pathogenic_variants': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/emptyRemoved_20160428_pathogenic_variants_HGVSg_VEP_grch38lifOver.vcf'
    },
    'charger_hotspot3d_clusters': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/cptac_mc3_combined_noHypers_sorted.maf.3D_Proximity.pairwise.recurrence.l0.ad10.r10.clusters'
    },
    'charger_clinvar_alleles': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/clinvar_alleles.single.b38.tsv.gz'
    },
}

In [32]:
yaml.safe_dump(pecgs_storage1_defaults, open('/home/estorrs/pecgs-pipeline/cwl/pecgs_workflows/defaults.pecgs_TN_wxs_fq.yaml', 'w'))

In [33]:
yaml.safe_dump(pecgs_storage1_defaults, open('../wombat/templates/compute1.defaults.pecgs_TN_wxs_fq.yaml', 'w'))

In [34]:
pecgs_storage1_defaults = {
    # general
    'cpu': 40,
    
    'reference': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'}
        ]
    },
    
    # cnv
    'common_biallelic': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
    },
    'pool_of_normals': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5'
    },
    'protein_coding_gene': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed'
    },
    'reference_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1'
    },
    'target_interval_list': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list'
    },
    
    # msisensor
    'microsatellite': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/hg38.microsatellite'
    },
    'minimal_homopolymer_size': 1,
    'minimal_microsatellite_size': 1,
    
    # tindaisy
    'clinvar_annotation': {
        'class': 'File',
        'path': f'{resources_dir}/clinvar/GRCh38/clinvar_20200706.vcf.gz'
    },
    'rescue_clinvar': True,
    'rescue_cosmic': True,
    'call_regions': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.callRegions.bed.gz'
    },
    'canonical_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.callRegions.bed'
    },
    'pindel_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/pindel.WES.ini'
    },
    'strelka_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/strelka.WES.ini'
    },
    'varscan_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/caller_config/varscan.WES.ini'
    },
    'classification_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/filter_config/classification_filter_config.ini'
    },
    'af_config': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/filter_config/af_filter_config.ini'
    },
    'centromere_bed': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/centromere/ucsc-centromere.GRCh38.bed'
    },
    'tindaisy_vep_cache_gz': {
        'class': 'File',
        'path': f'{resources_dir}/vep/v99/vep-cache.99_GRCh38.tar.gz'
    },
    'tindaisy_vep_cache_version': 99,
    'assembly': 'GRCh38',
    'tindaisy_chrlist': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/TinDaisy/params/chrlist/GRCh38.d1.vd1.chrlist.txt'
    },
    
    # TinJasmine
    'centromere': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/centromere/ucsc-centromere.GRCh38.bed'
    },
    'tinjasmine_chrlist': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/chrlist/GRCh38.d1.vd1.chrlist-reordered.txt'
    },
    'pindel_config_template': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_config_template/pindel_germline_filter_config.ini'
    },
    'Canonical_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/Canonical_BED/GRCh38.callRegions.bed'
    },
    'ROI_BED': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/ROI_BED/Homo_sapiens.GRCh38.95.allCDS.2bpFlanks.biomart.withCHR.bed'
    },
#     'varscan_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/varscan_filter_config/VLD_FilterVCF-varscan.config.ini'
#     },
#     'pindel_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/pindel_filter_config/VLD_FilterVCF-pindel.config.ini'
#     },
#     'gatk_filter_config': {
#         'class': 'File',
#         'path': '/storage1/fs1/dinglab/Active/Resources/Pipelines/TinJasmine/TinJasmine_etc/gatk_filter_config/VLD_FilterVCF-GATK.config.ini'
#     },
    'tinjasmine_vep_cache_gz': {
        'class': 'File',
        'path': f'{resources_dir}/vep/20220525/vep-cache.100_GRCh38.tar.gz'
    },
    
    # neoscan
    'neoscan_ref_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/neoscan/refseq_hg38_june29'
    },
    'neoscan_bed': {
        'class': 'File',
        'path': f'{resources_dir}/neoscan/refseq_hg38_june29/proteome.bed'
    },
    
    # charger
    'charger_inheritance_gene_list': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/cancer_pred_genes_160genes_011321_curated_forCharGer.txt'
    },
    'charger_pp2_gene_list': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/160cpgs.txt'
    },
    'charger_pathogenic_variants': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/emptyRemoved_20160428_pathogenic_variants_HGVSg_VEP_grch38lifOver.vcf'
    },
    'charger_hotspot3d_clusters': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/PanCan/cptac_mc3_combined_noHypers_sorted.maf.3D_Proximity.pairwise.recurrence.l0.ad10.r10.clusters'
    },
    'charger_clinvar_alleles': {
        'class': 'File',
        'path': f'{resources_dir}/charger/CharGer_dependencyFiles/clinvar_alleles.single.b38.tsv.gz'
    },
}

In [35]:
yaml.safe_dump(pecgs_storage1_defaults, open('/home/estorrs/pecgs-pipeline/cwl/pecgs_workflows/defaults.pecgs_TN_wxs_bam.yaml', 'w'))

In [36]:
yaml.safe_dump(pecgs_storage1_defaults, open('../wombat/templates/compute1.defaults.pecgs_TN_wxs_bam.yaml', 'w'))

In [17]:
pecgs_storage1_defaults = {
    # general
    'cpu': 40,
    
    # fusion
    'bwts': {
        'class': 'Directory',
        'path': f'{resources_dir}/fusion/Integrate_dependencies/bwts'
    },
    'filter_database': {
        'class': 'Directory',
        'path': f'{resources_dir}/fusion/FilterDatabase'
    },
    'fusion_annotator_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/fusion/FusionAnnotator'
    },
    'genome_db': {
        'class': 'Directory',
        'path': f'{resources_dir}/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84'
    },
    'genome_lib_dir': {
        'class': 'Directory',
        'path': f'{resources_dir}/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir'
    },
    'integrate_annotations': {
        'class': 'File',
        'path': f'{resources_dir}/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt'
    },
    'integrate_executable': {
        'class': 'File',
        'path': f'{resources_dir}/fusion/INTEGRATE_0_2_6/INTEGRATE-build/bin/Integrate'
    },
    'integrate_fasta': {
        'class': 'File',
        'path': f'{resources_dir}/fusion/Integrate_dependencies/STAR/hg38.fa'
    },
    
    # bulk expression
    'star_index': {
        'class': 'Directory',
        'path': f'{resources_dir}/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34'
    },
    'gtf': {
        'class': 'File',
        'path': f'{resources_dir}/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.v34.annotation.gtf'
    },
    'gene_info': {
        'class': 'File',
        'path': f'{resources_dir}/bulk_expression/STAR_GRCh38_d1_vd1_d1_GENCODE_v34/gencode.gene.info.v34.tsv'
    },
}

In [18]:
yaml.safe_dump(pecgs_storage1_defaults, open('/home/estorrs/pecgs-pipeline/cwl/pecgs_workflows/defaults.pecgs_T_rna_fq.yaml', 'w'))

In [19]:
yaml.safe_dump(pecgs_storage1_defaults, open('../wombat/templates/compute1.defaults.pecgs_T_rna_fq.yaml', 'w'))

In [20]:
pecgs_storage1_defaults = {
    # general
#     'cpu': 40,
    
    'reference': {
        'class': 'File',
        'path': f'{resources_dir}/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa',
    },
    
    # somatic sv
    'generate_evidence_bam': True,
}

In [21]:
yaml.safe_dump(pecgs_storage1_defaults, open('../wombat/templates/compute1.defaults.pecgs_TN_wgs_bam.yaml', 'w'))